# PyTorch MNIST Membership Inference demo

This notebook contains an end-to-end demostration of Dioptra that can be run on any modern laptop.

## Setup

Below we import the necessary Python modules and ensure the proper environment variables are set so that all the code blocks will work as expected,

In [ ]:
# Import packages from the Python standard library
import os
import pprint
import time
import warnings
from pathlib import Path
from typing import Tuple

# Filter out warning messages
warnings.filterwarnings("ignore")

# Address for connecting the docker container to exposed ports on the host device
HOST_DOCKER_INTERNAL = "host.docker.internal"
# HOST_DOCKER_INTERNAL = "172.17.0.1"

# Testbed API ports
RESTAPI_PORT = "30080"
MLFLOW_TRACKING_PORT = "35000"

# Default address for accessing the RESTful API service
RESTAPI_ADDRESS = (
    f"http://{HOST_DOCKER_INTERNAL}:{RESTAPI_PORT}"
    if os.getenv("IS_JUPYTER_SERVICE")
    else f"http://localhost:{RESTAPI_PORT}"
)

# Override the AI_RESTAPI_URI variable, used to connect to RESTful API service
os.environ["AI_RESTAPI_URI"] = RESTAPI_ADDRESS

# Default address for accessing the MLFlow Tracking server
MLFLOW_TRACKING_URI = (
    f"http://{HOST_DOCKER_INTERNAL}:{MLFLOW_TRACKING_PORT}"
    if os.getenv("IS_JUPYTER_SERVICE")
    else f"http://localhost:{MLFLOW_TRACKING_PORT}"
)

# Override the MLFLOW_TRACKING_URI variable, used to connect to MLFlow Tracking service
os.environ["MLFLOW_TRACKING_URI"] = MLFLOW_TRACKING_URI

# Base API address
RESTAPI_API_BASE = f"{RESTAPI_ADDRESS}/api"

# Path to workflows archive
WORKFLOWS_TAR_GZ = Path("workflows.tar.gz")

# Path to custom task plugins archive
CUSTOM_PLUGINS_PYTORCH_MI_TAR_GZ = Path("custom-plugins-pytorch-mi.tar.gz")

# Experiment name (note the username_ prefix convention)
EXPERIMENT_NAME = "jtsexton_inference_mnist"

# Path to dataset
data_path_mnist = "/nfs/data/Mnist"

# Import third-party Python packages
import numpy as np
import requests
from mlflow.tracking import MlflowClient

# Import utils.py file
import utils

# Create random number generator
rng = np.random.default_rng(54399264723942495723666216079516778448)

## Submit and run jobs

The entrypoints that we will be running in this example are implemented in the Python source files under `src/` and the `MLproject` file.
To run these entrypoints within the testbed architecture, we need to package those files up into an archive and submit it to the Testbed RESTful API to create a new job.
For convenience, the `Makefile` provides a rule for creating the archive file for this example, just run `make workflows`,

In [ ]:
%%bash

# Create the workflows.tar.gz file
make workflows

To connect with the endpoint, we will use a client class defined in the `utils.py` file that is able to connect with the Testbed RESTful API using the HTTP protocol.
We connect using the client below, which uses the environment variable `AI_RESTAPI_URI` to figure out how to connect to the Testbed RESTful API,

In [ ]:
restapi_client = utils.SecuringAIClient()

We need to register an experiment under which to collect our job runs.
The code below checks if the relevant experiment exists.
If it does, then it just returns info about the experiment, if it doesn't, it then registers the new experiment.

In [ ]:
response_experiment = restapi_client.get_experiment_by_name(name=EXPERIMENT_NAME)

if response_experiment is None or "Not Found" in response_experiment.get("message", []):
    response_experiment = restapi_client.register_experiment(name=EXPERIMENT_NAME)

response_experiment

We should also check which queues are available for running our jobs to make sure that the resources that we need are available.
The code below queries the Lab API and returns a list of active queues.

In [ ]:
restapi_client.list_queues()

This example also makes use of the `pytorch_mi` custom task plugin package stored locally under the `task-plugins/securingai_custom/pytorch_mi` directory.
To register these custom task plugins, we first need to package them up into an archive.
For convenience, the `Makefile` provides a rule for creating the custom task plugins archive file, just run `make custom-plugins`,

In [ ]:
%%bash

# Create the workflows.tar.gz file
make custom-plugins

Now that the `pytorch_mi` custom task plugin package is packaged into an archive file, next we register it by uploading the file to the REST API.
Note that we need to provide the name to use for custom task plugin package, this name must be unique under the custom task plugins namespace.
For a full list of the custom task plugins, use `restapi_client.restapi_client.list_custom_task_plugins()`.

In [ ]:
response_custom_plugins = restapi_client.get_custom_task_plugin(name="pytorch_mi")
#restapi_client.delete_custom_task_plugin(name="pytorch_mi")
if response_custom_plugins is None or "Not Found" in response_custom_plugins.get("message", []):
    response_custom_plugins = restapi_client.upload_custom_plugin_package(
        custom_plugin_name="pytorch_mi",
        custom_plugin_file=CUSTOM_PLUGINS_PYTORCH_MI_TAR_GZ,
    )

response_custom_plugins

If at any point you need to update one or more files within the `pytorch_mi` plugin package, you will need to unregister/delete the custom task plugin first using the REST API.
This can be done as follows,

In [ ]:

# Delete the 'pytorch_mi' custom task plugin package
restapi_client.delete_custom_task_plugin(name="pytorch_mi")


After you have deleted the task plugin in the testbed, re-run the `make custom-plugins` code block to update the package archive, then upload the updated plugin by re-running the `restapi_client.upload_custom_plugin_packge` block.

Next, we need to train our model.
We will be using the V100 GPUs that are available on the DGX Workstation, which we can use by submitting our job to the `"tensorflow_gpu"` queue.
We will train one LeNet-5 model for use in the membership inference attack.


In [ ]:
response_le_net_train = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="train",
    entry_point_kwargs=" ".join([
        "-P data_dir=/nfs/data/Mnist",
        "-P register_model_name=le_net",
        "-P batch_size=256",
        "-P register_model=True",
        "-P model_architecture=le_net",
        "-P epochs=30",
        f"-P data_dir={data_path_mnist}",
    ]),
    queue="pytorch_gpu",
    timeout="1h",
)

print("Training job for LeNet-5 neural network submitted")
print("")
pprint.pprint(response_le_net_train)

Now that we have an MNIST trained model, we can run the Membership Inference attack on it. The Membership Inference attack essentially trains a classifier on training and non-training data for the original model. It then uses this model to predict whether new data was part of the training set or not.
* batch_size - how many samples to try to use from the training/testing set
* balance_sets - if True, this will truncate the bigger of the training/testing set such that the sets have the same number of members
* attack_model_type - 'nn', 'gb' or 'rf', for neural network, gradient boosting, or random forests. this decides the type of classifier used in the membership inference attack.
* split - decimal value representing the percentage of the available data which should be used for training.


In [ ]:
def mlflow_run_id_is_not_known(response_infer):
    return response_infer["mlflowRunId"] is None and response_infer["status"] not in [
        "failed",
        "finished",
    ]

response_infer = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="mi",
    entry_point_kwargs=" ".join(
        [
            f"-P model_name=le_net",
            "-P model_version=1",
            "-P data_dir=/nfs/data/Mnist",
            "-P batch_size=50000",
            "-P attack_model_type=nn",
            "-P balance_sets=False",
            "-P split=0.2"
        ]
    ),
    queue="pytorch_gpu",
)

print("INFER attack (LE net architecture) job submitted!!")
print("")
print("")

while mlflow_run_id_is_not_known(response_infer):
    time.sleep(1)
    response_infer = restapi_client.get_job_by_id(
        response_infer["jobId"]
    )